In [1]:
from utils import *

# 数据处理

## 用户特征处理  
> (1)对部分特征编码  
>- age：作为有序多分类特征，保持原样。
>- gender: 无序多分类特征，采用one-hot编码
>- country：无序多分类特征，且分类特别少，采用one-hot编码
>- provience：无序多分类特征，分类数较多，采用Frequency编码
>- city：无序多分类特征，分类数较多，采用target编码
>- city_level：有序多分类特征，保持。
>- device_name：无序多分类特征，分类数较多，采用target编码。   

> (2)追加部分统计特征(以1,3,7,14天为时间窗口)  
>- 期限内用户观看视频次数
>- 期限内用户观看视频部数
>- 用户评论视频次数
>- 用户收藏视频次数
>- 用户分享视频数量

> (3)对追加的统计特征做数据平滑,以减小统计的误差。

In [2]:
user_df = load_user()

In [3]:
user_df

,user_id,age,gender,country,province,city,city_level,device_name
0,1757005,3,1,0,9,6,3,327
1,17938,0,0,0,4,22,3,327
2,4263520,1,0,0,19,1,5,327
3,1411600,3,0,0,5,138,1,327
4,3992242,2,0,0,0,142,0,327
...,...,...,...,...,...,...,...,...
5910795,3223427,4,0,0,3,3,3,28
5910796,4707826,4,0,0,17,249,1,28
5910797,5907653,0,0,0,11,65,0,28
5910798,3633224,3,0,0,2,57,1,28


### 特征编码

#### age
>原数据为分段编码，有序多分类，保持原样。

#### gender
>性别，采用one-hot编码

In [4]:
col_name = 'gender'
df = user_df[col_name]
new_cols = pd.get_dummies(df)
new_col_names = new_cols.columns.values
new_cols.columns = [f"{col_name}_{col}" for col in new_col_names]
user_df = pd.concat([user_df, new_cols], axis=1)
user_df.drop([col_name], axis=1, inplace=True)
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5910800 entries, 0 to 5910799
Data columns (total 11 columns):
 #   Column       Dtype
---  ------       -----
 0   user_id      int32
 1   age          int32
 2   country      int32
 3   province     int32
 4   city         int32
 5   city_level   int32
 6   device_name  int32
 7   gender_0     uint8
 8   gender_1     uint8
 9   gender_2     uint8
 10  gender_3     uint8
dtypes: int32(7), uint8(4)
memory usage: 180.4 MB


#### country
>无序多分类特征，且分类只有3个，采用one-hot编码

In [5]:
col_name = 'country'
df = user_df[col_name]
new_cols = pd.get_dummies(df)
new_col_names = new_cols.columns.values
new_cols.columns = [f"{col_name}_{col}" for col in new_col_names]
user_df = pd.concat([user_df, new_cols], axis=1)
user_df.drop([col_name], axis=1, inplace=True)
user_df.head()

,user_id,age,province,city,city_level,device_name,gender_0,gender_1,gender_2,gender_3,country_0,country_1,country_2
0,1757005,3,9,6,3,327,0,1,0,0,1,0,0
1,17938,0,4,22,3,327,1,0,0,0,1,0,0
2,4263520,1,19,1,5,327,1,0,0,0,1,0,0
3,1411600,3,5,138,1,327,1,0,0,0,1,0,0
4,3992242,2,0,142,0,327,1,0,0,0,1,0,0


In [6]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5910800 entries, 0 to 5910799
Data columns (total 13 columns):
 #   Column       Dtype
---  ------       -----
 0   user_id      int32
 1   age          int32
 2   province     int32
 3   city         int32
 4   city_level   int32
 5   device_name  int32
 6   gender_0     uint8
 7   gender_1     uint8
 8   gender_2     uint8
 9   gender_3     uint8
 10  country_0    uint8
 11  country_1    uint8
 12  country_2    uint8
dtypes: int32(6), uint8(7)
memory usage: 174.7 MB


#### provience
>无需多分类特征，分类数较多，所以采用频率编码，再降维,Frequency编码通过计算特征变量中每个值的出现次数来表示该特征的信息。

In [7]:
user_df

,user_id,age,province,city,city_level,device_name,gender_0,gender_1,gender_2,gender_3,country_0,country_1,country_2
0,1757005,3,9,6,3,327,0,1,0,0,1,0,0
1,17938,0,4,22,3,327,1,0,0,0,1,0,0
2,4263520,1,19,1,5,327,1,0,0,0,1,0,0
3,1411600,3,5,138,1,327,1,0,0,0,1,0,0
4,3992242,2,0,142,0,327,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5910795,3223427,4,3,3,3,28,1,0,0,0,1,0,0
5910796,4707826,4,17,249,1,28,1,0,0,0,1,0,0
5910797,5907653,0,11,65,0,28,1,0,0,0,1,0,0
5910798,3633224,3,2,57,1,28,1,0,0,0,1,0,0


In [8]:
col_name = 'province'
user_df[col_name] = user_df[col_name].map(user_df[col_name].value_counts())

In [9]:
user_df.head()

,user_id,age,province,city,city_level,device_name,gender_0,gender_1,gender_2,gender_3,country_0,country_1,country_2
0,1757005,3,229863,6,3,327,0,1,0,0,1,0,0
1,17938,0,341554,22,3,327,1,0,0,0,1,0,0
2,4263520,1,120423,1,5,327,1,0,0,0,1,0,0
3,1411600,3,314573,138,1,327,1,0,0,0,1,0,0
4,3992242,2,630065,142,0,327,1,0,0,0,1,0,0


#### city
>无序多分类特征，分类数达339，采用frequency

In [10]:
col_name = 'city'
user_df[col_name] = user_df[col_name].map(user_df[col_name].value_counts())
user_df.head()

,user_id,age,province,city,city_level,device_name,gender_0,gender_1,gender_2,gender_3,country_0,country_1,country_2
0,1757005,3,229863,96162,3,327,0,1,0,0,1,0,0
1,17938,0,341554,45486,3,327,1,0,0,0,1,0,0
2,4263520,1,120423,120423,5,327,1,0,0,0,1,0,0
3,1411600,3,314573,14566,1,327,1,0,0,0,1,0,0
4,3992242,2,630065,14379,0,327,1,0,0,0,1,0,0


#### city_level
>有序多分类变量，保持

#### device_name
>无序多分类变量，类别基数大，采用target编码:LeaveOneOutEncoder  
参考资料：https://axk51013.medium.com/kaggle-categorical-encoding-3%E5%A4%A7%E7%B5%95%E6%8B%9B-589780119470

- 需采用交叉验证，目前用Frequency代替
col_name = 'device_name'
loo = LeaveOneOutEncoder()
loo.fit_transform(df_tr['color'], df_tr['label'])

In [11]:
user_df

,user_id,age,province,city,city_level,device_name,gender_0,gender_1,gender_2,gender_3,country_0,country_1,country_2
0,1757005,3,229863,96162,3,327,0,1,0,0,1,0,0
1,17938,0,341554,45486,3,327,1,0,0,0,1,0,0
2,4263520,1,120423,120423,5,327,1,0,0,0,1,0,0
3,1411600,3,314573,14566,1,327,1,0,0,0,1,0,0
4,3992242,2,630065,14379,0,327,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5910795,3223427,4,343991,113444,3,28,1,0,0,0,1,0,0
5910796,4707826,4,138629,6908,1,28,1,0,0,0,1,0,0
5910797,5907653,0,206316,23276,0,28,1,0,0,0,1,0,0
5910798,3633224,3,437480,25884,1,28,1,0,0,0,1,0,0


使用Frequency方法代替Leave_one_out对device_name编码。

In [12]:
col_name = 'device_name'
user_df[col_name] = user_df[col_name].map(user_df[col_name].value_counts())
user_df.head()

,user_id,age,province,city,city_level,device_name,gender_0,gender_1,gender_2,gender_3,country_0,country_1,country_2
0,1757005,3,229863,96162,3,1022,0,1,0,0,1,0,0
1,17938,0,341554,45486,3,1022,1,0,0,0,1,0,0
2,4263520,1,120423,120423,5,1022,1,0,0,0,1,0,0
3,1411600,3,314573,14566,1,1022,1,0,0,0,1,0,0
4,3992242,2,630065,14379,0,1022,1,0,0,0,1,0,0


In [13]:
# save_user_data(user_df, "user_data_v2", "jay")

## 统计特征追加 

In [2]:
from utils import * 

In [3]:
%%time
actions_df = load_actions(all_features = True)

CPU times: user 25.3 s, sys: 1.66 s, total: 27 s
Wall time: 7.94 s


In [4]:
actions_df.head()

,user_id,video_id,is_watch,is_share,is_collect,is_comment,watch_start_time,watch_label,pt_d
0,3672407,38350,False,False,False,False,NaN,0,20210427
1,3080901,11907,False,False,False,False,NaN,0,20210427
2,3528503,28411,False,False,False,False,NaN,0,20210427
3,3528503,15070,False,False,False,False,NaN,0,20210427
4,3528503,38350,False,False,False,False,NaN,0,20210427


### 以用户分类

In [5]:
# 加载用户原表
user_df = load_user()

In [6]:
user_groups = actions_df.groupby(['user_id'])

In [7]:
user_status = pd.DataFrame(user_df['user_id'])

In [8]:
user_status

,user_id
0,1757005
1,17938
2,4263520
3,1411600
4,3992242
...,...
5910795,3223427
5910796,4707826
5910797,5907653
5910798,3633224


#### 用户14天平均watch_label

In [9]:
average_watch_label = user_groups['watch_label'].agg(['mean'])

In [10]:
average_watch_label = pd.DataFrame(average_watch_label)

In [11]:
average_watch_label = renameCol(average_watch_label, 'mean', 'average_watch_label')

In [12]:
average_watch_label

,average_watch_label
user_id,
2,0.368421
4,0.538462
5,0.000000
6,0.000000
7,0.269231
...,...
5910793,0.000000
5910794,0.029412
5910795,0.000000


In [13]:
user_status = pd.merge(user_status, average_watch_label, on='user_id', how='left')
user_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5910800 entries, 0 to 5910799
Data columns (total 2 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_id              int32  
 1   average_watch_label  float64
dtypes: float64(1), int32(1)
memory usage: 112.7 MB


#### 用户14天观看视频次数

In [14]:
sum_watch_times = user_groups['is_watch'].agg(['sum'])

In [15]:
sum_watch_times = pd.DataFrame(sum_watch_times)

In [16]:
sum_watch_times = renameCol(sum_watch_times, 'sum', 'sum_watch_times')

In [17]:
sum_watch_times

,sum_watch_times
user_id,
2,15
4,1
5,0
6,0
7,2
...,...
5910793,1
5910794,2
5910795,0


In [18]:
user_status = pd.merge(user_status, sum_watch_times, on='user_id', how='left')
user_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5910800 entries, 0 to 5910799
Data columns (total 3 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_id              int32  
 1   average_watch_label  float64
 2   sum_watch_times      float64
dtypes: float64(2), int32(1)
memory usage: 157.8 MB


#### 用户14天完播视频次数
> 完播：watch_label>=8，播放80%视频时间。  
! 太慢了，先不做

In [ ]:
user_groups['watch_label'].apply(lambda x: sum(x >= 8))

In [ ]:
sum(user_groups['watch_label'][:]>=8)

In [ ]:
user_groups['watch_label'].apply(lambda x: sum(x >= 8))

#### 用户14天跳过视频次数
> watch_label = 0

In [ ]:
user_groups['watch_label'].apply(lambda x: sum(x == 0))

#### 用户14天评论视频次数

In [19]:
sum_comment_times = user_groups['is_comment'].agg(['sum'])

In [20]:
sum_comment_times = pd.DataFrame(sum_comment_times)

In [21]:
sum_comment_times = renameCol(sum_comment_times, 'sum', 'sum_comment_times')

In [22]:
sum_comment_times

,sum_comment_times
user_id,
2,0
4,0
5,0
6,0
7,0
...,...
5910793,0
5910794,0
5910795,0


In [23]:
user_status = pd.merge(user_status, sum_comment_times, on='user_id', how='left')
user_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5910800 entries, 0 to 5910799
Data columns (total 4 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_id              int32  
 1   average_watch_label  float64
 2   sum_watch_times      float64
 3   sum_comment_times    float64
dtypes: float64(3), int32(1)
memory usage: 202.9 MB


#### 用户14天收藏视频次数

In [24]:
sum_collect_times = user_groups['is_collect'].agg(['sum'])

In [25]:
sum_collect_times = pd.DataFrame(sum_collect_times)

In [26]:
sum_collect_times = renameCol(sum_collect_times, 'sum', 'sum_collect_times')

In [27]:
sum_collect_times

,sum_collect_times
user_id,
2,0
4,0
5,0
6,0
7,0
...,...
5910793,0
5910794,0
5910795,0


In [28]:
user_status = pd.merge(user_status, sum_collect_times, on='user_id', how='left')
user_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5910800 entries, 0 to 5910799
Data columns (total 5 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_id              int32  
 1   average_watch_label  float64
 2   sum_watch_times      float64
 3   sum_comment_times    float64
 4   sum_collect_times    float64
dtypes: float64(4), int32(1)
memory usage: 248.0 MB


#### 用户14天分享视频次数

In [29]:
sum_share_times = user_groups['is_share'].agg(['sum'])

In [30]:
sum_share_times = pd.DataFrame(sum_share_times)

In [31]:
sum_share_times = renameCol(sum_share_times, 'sum', 'sum_share_times')

In [32]:
sum_share_times

,sum_share_times
user_id,
2,0
4,0
5,0
6,0
7,0
...,...
5910793,0
5910794,0
5910795,0


In [33]:
user_status = pd.merge(user_status, sum_share_times, on='user_id', how='left')
user_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5910800 entries, 0 to 5910799
Data columns (total 6 columns):
 #   Column               Dtype  
---  ------               -----  
 0   user_id              int32  
 1   average_watch_label  float64
 2   sum_watch_times      float64
 3   sum_comment_times    float64
 4   sum_collect_times    float64
 5   sum_share_times      float64
dtypes: float64(5), int32(1)
memory usage: 293.1 MB


#### 保存

In [35]:
user_status.fillna(0, inplace=True)

In [36]:
user_status

,user_id,average_watch_label,sum_watch_times,sum_comment_times,sum_collect_times,sum_share_times
0,1757005,0.000000,0.0,0.0,0.0,0.0
1,17938,0.096774,3.0,0.0,0.0,0.0
2,4263520,0.204545,2.0,0.0,0.0,0.0
3,1411600,0.000000,0.0,0.0,0.0,0.0
4,3992242,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
5910795,3223427,0.000000,0.0,0.0,0.0,0.0
5910796,4707826,0.142857,3.0,0.0,0.0,0.0
5910797,5907653,0.000000,2.0,0.0,0.0,0.0
5910798,3633224,0.000000,0.0,0.0,0.0,0.0


In [37]:
save_user_data(user_status, 'user_status')

### 以视频分类

In [2]:
video_df = load_video()

In [5]:
video_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50355 entries, 0 to 50354
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   video_name           50355 non-null  object 
 2   video_tags           50355 non-null  object 
 3   video_description    50355 non-null  object 
 4   video_release_date   50355 non-null  object 
 5   video_director_list  50355 non-null  object 
 6   video_actor_list     50355 non-null  object 
 7   video_score          46994 non-null  float64
 8   video_second_class   50355 non-null  object 
 9   video_duration       50355 non-null  int32  
dtypes: float64(1), int32(2), object(7)
memory usage: 3.5+ MB


In [3]:
video_status = pd.DataFrame(video_df['video_id'])

In [4]:
video_status.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50355 entries, 0 to 50354
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   video_id  50355 non-null  int32
dtypes: int32(1)
memory usage: 196.8 KB


In [7]:
video_groups = actions_df.groupby(['video_id'])

#### 视频14天平均watch_label

In [8]:
average_watch_label = video_groups['watch_label'].agg('mean')

In [9]:
average_watch_label = couerGroupToDF(average_watch_label, 'mean', 'average_watch_label')

In [10]:
average_watch_label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34218 entries, 0 to 50354
Data columns (total 1 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   average_watch_label  34218 non-null  float64
dtypes: float64(1)
memory usage: 534.7 KB


In [11]:
video_status = pd.merge(video_status, average_watch_label, on='video_id', how='left')
video_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50355 entries, 0 to 50354
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   average_watch_label  34218 non-null  float64
dtypes: float64(1), int32(1)
memory usage: 983.5 KB


In [12]:
video_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50355 entries, 0 to 50354
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   average_watch_label  34218 non-null  float64
dtypes: float64(1), int32(1)
memory usage: 983.5 KB


#### 视频14天被观看次数

In [13]:
sum_watch_times = video_groups['is_watch'].apply(lambda x: sum(x == 1))

In [14]:
sum_watch_times = couerGroupToDF(sum_watch_times, 'sum', 'sum_watch_times')

In [15]:
video_status = pd.merge(video_status, sum_watch_times, on='video_id', how='left')
video_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50355 entries, 0 to 50354
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   average_watch_label  34218 non-null  float64
 2   sum_watch_times      34218 non-null  float64
dtypes: float64(2), int32(1)
memory usage: 1.3 MB


#### 视频14天完播数
> 完播次数/总播放次数
> 完播放（watch_label >= 8）

In [16]:
watch_over_times = video_groups['watch_label'].apply(lambda x: sum(x >= 8))

In [17]:
watch_over_times = couerGroupToDF(watch_over_times, 'sum', 'watch_over_times')

In [18]:
watch_over_times.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34218 entries, 0 to 50354
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   watch_over_times  34218 non-null  int64
dtypes: int64(1)
memory usage: 534.7 KB


In [19]:
video_status = pd.merge(video_status, watch_over_times, on='video_id', how='left')
video_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50355 entries, 0 to 50354
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   average_watch_label  34218 non-null  float64
 2   sum_watch_times      34218 non-null  float64
 3   watch_over_times     34218 non-null  float64
dtypes: float64(3), int32(1)
memory usage: 1.7 MB


#### 视频14天跳过数
> 跳过（watch_label = 0）

In [20]:
sum_skip_times = video_groups['watch_label'].apply(lambda x: sum(x == 0))

In [21]:
sum_skip_times = couerGroupToDF(sum_skip_times, 'sum_skip_times','sum_skip_times')

In [22]:
sum_skip_times.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34218 entries, 0 to 50354
Data columns (total 1 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   sum_skip_times  34218 non-null  int64
dtypes: int64(1)
memory usage: 534.7 KB


In [23]:
video_status = pd.merge(video_status, sum_skip_times, on='video_id', how='left')
video_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50355 entries, 0 to 50354
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   average_watch_label  34218 non-null  float64
 2   sum_watch_times      34218 non-null  float64
 3   watch_over_times     34218 non-null  float64
 4   sum_skip_times       34218 non-null  float64
dtypes: float64(4), int32(1)
memory usage: 2.1 MB


#### 视频14天评论数

In [30]:
comments_times = video_groups['is_comment'].agg('sum')

In [31]:
comments_times = couerGroupToDF(comments_times, '', 'comments_times')

In [32]:
video_status = pd.merge(video_status, comments_times, on='video_id', how='left')
video_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50355 entries, 0 to 50354
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   average_watch_label  34218 non-null  float64
 2   sum_watch_times      34218 non-null  float64
 3   watch_over_times     34218 non-null  float64
 4   sum_skip_times       34218 non-null  float64
 5   comments_times       34218 non-null  float64
dtypes: float64(5), int32(1)
memory usage: 2.5 MB


#### 视频14天收藏数

In [33]:
collect_times = video_groups['is_collect'].agg('sum')

In [34]:
collect_times = couerGroupToDF(collect_times, 'collect_times', 'collect_times')

In [35]:
collect_times.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34218 entries, 0 to 50354
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   collect_times  34218 non-null  int64
dtypes: int64(1)
memory usage: 534.7 KB


In [36]:
video_status = pd.merge(video_status, collect_times, on='video_id', how='left')
video_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50355 entries, 0 to 50354
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   average_watch_label  34218 non-null  float64
 2   sum_watch_times      34218 non-null  float64
 3   watch_over_times     34218 non-null  float64
 4   sum_skip_times       34218 non-null  float64
 5   comments_times       34218 non-null  float64
 6   collect_times        34218 non-null  float64
dtypes: float64(6), int32(1)
memory usage: 2.9 MB


#### 视频14天分享数

In [37]:
share_times = video_groups['is_share'].apply(lambda x: sum(x==1))

In [38]:
share_times = couerGroupToDF(share_times,'share_times','share_times')

In [39]:
share_times

,share_times
video_id,
0,0
2,0
3,0
4,0
5,0
...,...
50344,0
50345,0
50347,0


In [40]:
video_status = pd.merge(video_status, share_times, on='video_id', how='left')
video_status.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50355 entries, 0 to 50354
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   video_id             50355 non-null  int32  
 1   average_watch_label  34218 non-null  float64
 2   sum_watch_times      34218 non-null  float64
 3   watch_over_times     34218 non-null  float64
 4   sum_skip_times       34218 non-null  float64
 5   comments_times       34218 non-null  float64
 6   collect_times        34218 non-null  float64
 7   share_times          34218 non-null  float64
dtypes: float64(7), int32(1)
memory usage: 3.3 MB


#### 保存

In [44]:
video_status.fillna(0, inplace=True)

In [45]:
save_video_temp_data(video_status, 'video_status')

保存成功
保存路径为： ../../dataset/traindata/video_features_data/video_status.csv


## 按时间统计
> 主要工作为统计每天中每个用户，每只视频的相关数据
>1. 用户
>- 用户平均观看时长比例（ avg watch_label ）
>- 观看视频次数（action中is_watch=1的数目）
>- 观看视频部数（action中is_watch=1的视频个数，去重*）
>- 用户完播视频次数（watch_label >= 8）
>- 用户退出次数（is_watch=1, watch_label=0）
>- 用户划过次数（is_watch=0）
>- 评论、收藏、分享次数
>- 观看时长 ( sum(watch_label * video_duration) )
>2. 视频
>- 完播次数（watch_lable >= 8）
>- 被观看次数（is_watch=1）
>- 评论、收藏、分享次数
>- 退出次数（is_watch=1, watch_label=0）
>- 划过次数（is_watch=0）
>- 平均观看时长（avg watch_label）

- 加载14天全量数据集
> dataset/traindata/history_behavior_data/all_action_features.csv

In [81]:
action_df = load("../../dataset/traindata/history_behavior_data/all_action_features.csv")

In [82]:
action_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80276856 entries, 0 to 80276855
Data columns (total 9 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           int32 
 1   video_id          int32 
 2   is_watch          bool  
 3   is_share          bool  
 4   is_collect        bool  
 5   is_comment        bool  
 6   watch_start_time  object
 7   watch_label       int32 
 8   pt_d              int32 
dtypes: bool(4), int32(4), object(1)
memory usage: 2.1+ GB


In [83]:
dtypes = action_df.dtypes

In [84]:
dtypes['is_watch', 'is_share', 'is_collect', 'is_comment'] = 'int32'

In [85]:
action_df = action_df.astype(dtypes)

In [86]:
action_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80276856 entries, 0 to 80276855
Data columns (total 9 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           int32 
 1   video_id          int32 
 2   is_watch          int32 
 3   is_share          int32 
 4   is_collect        int32 
 5   is_comment        int32 
 6   watch_start_time  object
 7   watch_label       int32 
 8   pt_d              int32 
dtypes: int32(8), object(1)
memory usage: 3.0+ GB


In [87]:
action_df.head()

,user_id,video_id,is_watch,is_share,is_collect,is_comment,watch_start_time,watch_label,pt_d
0,3672407,38350,0,0,0,0,NaN,0,20210427
1,3080901,11907,0,0,0,0,NaN,0,20210427
2,3528503,28411,0,0,0,0,NaN,0,20210427
3,3528503,15070,0,0,0,0,NaN,0,20210427
4,3528503,38350,0,0,0,0,NaN,0,20210427


In [88]:
action_df = action_df[action_df['is_watch'] == 1]

In [89]:
action_df

,user_id,video_id,is_watch,is_share,is_collect,is_comment,watch_start_time,watch_label,pt_d
38447,4330891,16901,1,0,0,0,2021-04-27,0,20210427
38448,4330891,16813,1,0,0,0,2021-04-27,0,20210427
38449,2828470,11926,1,0,0,0,2021-04-27,0,20210427
38450,3594099,38350,1,0,0,0,2021-04-27,0,20210427
38451,4971201,44786,1,0,0,0,2021-04-27,0,20210427
...,...,...,...,...,...,...,...,...,...
80276851,1578540,6693,1,0,0,0,2021-05-01,0,20210501
80276852,3241301,3468,1,0,0,0,2021-05-01,0,20210501
80276853,2193774,28149,1,0,0,0,2021-05-01,0,20210501
80276854,2193774,7191,1,0,0,0,2021-05-01,9,20210501


- 要将video时长信息拼接起来
> dataset/traindata/video_features_data/video_features_data.csv

In [90]:
video_duration = dt.fread("../../dataset/traindata/video_features_data/new_video_features.jay").to_pandas()

In [91]:
video_duration = video_duration[['video_id', 'video_duration']]

In [92]:
video_duration

,video_id,video_duration
0,3460.0,5913.0
1,14553.0,6217.0
2,1214.0,5963.0
3,30639.0,17371.0
4,38522.0,10608.0
...,...,...
50350,36024.0,5736.0
50351,11306.0,6034.0
50352,16178.0,6918.0
50353,5337.0,6885.0


In [93]:
# video_duration = pd.read_csv("../../dataset/traindata/video_features_data/video_features_data.csv",sep="\t", usecols=['video_id', 'video_duration'])

In [94]:
# video_duration

In [95]:
action_df = pd.merge(action_df, video_duration, on='video_id', how='left')

In [96]:
action_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7353024 entries, 0 to 7353023
Data columns (total 10 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           int32  
 1   video_id          int32  
 2   is_watch          int32  
 3   is_share          int32  
 4   is_collect        int32  
 5   is_comment        int32  
 6   watch_start_time  object 
 7   watch_label       int32  
 8   pt_d              int32  
 9   video_duration    float32
dtypes: float32(1), int32(8), object(1)
memory usage: 364.6+ MB


In [97]:
u_list = np.sort(np.array(action_df['user_id'].unique()))
v_list = np.sort(np.array(action_df['video_id'].unique()))
d_list = np.sort(np.array(action_df['pt_d'].unique()))

In [98]:
d_list = list(d_list)

In [99]:
d_list

[20210419,
 20210420,
 20210421,
 20210422,
 20210423,
 20210424,
 20210425,
 20210426,
 20210427,
 20210428,
 20210429,
 20210430,
 20210501,
 20210502]

In [100]:
d_list.append(20210418)
d_list.append(20210503)

In [101]:
d_list = sorted(d_list)

In [102]:
print(d_list)

[20210418, 20210419, 20210420, 20210421, 20210422, 20210423, 20210424, 20210425, 20210426, 20210427, 20210428, 20210429, 20210430, 20210501, 20210502, 20210503]


In [103]:
from datatable import (dt, f, by, ifelse, update, sort,
                       count, min, max, mean, sum, rowsum, unique)

df =  dt.Frame(action_df)

### 用户
>- 用户平均观看时长比例（ avg watch_label ）
>- 观看视频次数（action中is_watch=1的数目）
>- 观看视频部数（action中is_watch=1的视频个数，去重*）
>- 用户完播视频次数（watch_label >= 8）
>- 用户退出次数（is_watch=1, watch_label=0）
>- 用户划过次数（is_watch=0）
>- 评论、收藏、分享次数
>- 观看时长 ( sum(watch_label * video_duration) )

In [48]:
%%time
user_all_status = df[:, {'u_avg_watch_label': mean(f.watch_label), 
       'u_sum_watch_times': sum(f.is_watch == 1), 
#         'sum_watch_videos': (f.is_watch == 1)             count(unique(df[f.is_watch == 1, 0][])),
       'u_sum_watch_overs': sum(f.watch_label >= 8),
        'u_sum_quit_times': sum((f.watch_label == 0) & (f.is_watch == 1)),
        'u_sum_skip_times': sum(f.is_watch == 0),
       'u_sum_comment_times': sum(f.is_comment == 1),
       'u_sum_collect_times': sum(f.is_collect == 1),
       'u_sum_share_times': sum(f.is_share == 1),
       'u_sum_watch_time': sum(f.watch_label * f.video_duration) / 10,
        'u_sum_watch_days': ifelse(sum(ifelse(f.is_watch == 1, 1, 0)) > 0, 1, 0)
      }, 
   by('user_id', 'pt_d')]

CPU times: user 6.04 s, sys: 0 ns, total: 6.04 s
Wall time: 504 ms


In [49]:
user_all_status = user_all_status.to_pandas()

In [50]:
user_all_status

,user_id,pt_d,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
0,2,20210419,3.000000,2,0,0,0,0,0,0,3455.699951,1
1,2,20210422,0.666667,3,0,2,0,0,0,0,969.200012,1
2,2,20210423,2.666667,6,0,1,0,0,0,0,11101.000000,1
3,2,20210424,0.000000,1,0,1,0,0,0,0,0.000000,1
4,2,20210429,2.000000,2,0,1,0,0,0,0,2064.399902,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4192648,5910793,20210425,0.000000,1,0,1,0,0,0,0,0.000000,1
4192649,5910794,20210424,1.000000,1,0,0,0,0,0,0,534.799988,1
4192650,5910794,20210425,0.000000,1,0,1,0,0,0,0,0.000000,1
4192651,5910797,20210502,0.000000,1,0,1,0,0,0,0,0.000000,1


- 将未出现的日期补零

In [51]:
save_user_data(user_all_status, 'user_all_status')

- 导出所有用户和时间的组合

In [55]:
user_all_status = dt.fread("../../dataset/traindata/user_features_data/user_all_status.csv")

In [56]:
# 保存结果
real_user_date = list(user_all_status[:, ['user_id','pt_d']].to_tuples())
all_user_date = [(u, d) for u in u_list for d in d_list] 

In [57]:
len(real_user_date)

4192653

In [58]:
len(all_user_date)

43051680

In [59]:
r_set = set(real_user_date)
a_set = set(all_user_date)

In [60]:
need_to_add = a_set - r_set

In [61]:
len(need_to_add)

38859027

In [62]:
del r_set, a_set

In [63]:
del all_user_date, real_user_date

In [64]:
need_to_add = list(need_to_add)

In [65]:
need_to_add[0]

(772533, 20210426)

In [66]:
tmp = [(u, d) for u,d in need_to_add ]

In [67]:
user_date_dtf = dt.Frame(tmp, names=['user_id', 'pt_d'])

In [68]:
del tmp,need_to_add

In [70]:
user_date_dtf

,user_id,pt_d,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,772533,20210426,0,0,0,0,0,0,0,0,0,0
1,2780928,20210419,0,0,0,0,0,0,0,0,0,0
2,480573,20210423,0,0,0,0,0,0,0,0,0,0
3,4224877,20210430,0,0,0,0,0,0,0,0,0,0
4,5327352,20210420,0,0,0,0,0,0,0,0,0,0
5,4398878,20210426,0,0,0,0,0,0,0,0,0,0
6,3610362,20210427,0,0,0,0,0,0,0,0,0,0
7,465050,20210423,0,0,0,0,0,0,0,0,0,0
8,5671156,20210430,0,0,0,0,0,0,0,0,0,0


 	u_avg_watch_label 	u_sum_watch_times 	u_sum_watch_overs 	u_sum_quit_times 	u_sum_skip_times 	u_sum_comment_times 	u_sum_collect_times 	u_sum_share_times 	u_sum_watch_time

In [71]:
user_date_dtf['u_avg_watch_label'] = 0
user_date_dtf['u_sum_watch_times'] = 0
user_date_dtf['u_sum_watch_overs'] = 0
user_date_dtf['u_sum_quit_times'] = 0
user_date_dtf['u_sum_skip_times'] = 0
user_date_dtf['u_sum_comment_times'] = 0
user_date_dtf['u_sum_collect_times'] = 0
user_date_dtf['u_sum_share_times'] = 0
user_date_dtf['u_sum_watch_time'] = 0
user_date_dtf['u_sum_watch_days'] = 0

In [72]:
user_all_status.names == user_date_dtf.names

True

In [73]:
all_user_date_dtf = dt.rbind(user_all_status, user_date_dtf, bynames = True)

In [74]:
all_user_date_dtf

,user_id,pt_d,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2,20210419,3,2,0,0,0,0,0,0,3455.7,1
1,2,20210422,0.666667,3,0,2,0,0,0,0,969.2,1
2,2,20210423,2.66667,6,0,1,0,0,0,0,11101,1
3,2,20210424,0,1,0,1,0,0,0,0,0,1
4,2,20210429,2,2,0,1,0,0,0,0,2064.4,1
5,2,20210502,0,1,0,1,0,0,0,0,0,1
6,4,20210426,7,1,0,0,0,0,0,0,3572.1,1
7,7,20210421,5,1,0,0,0,0,0,0,1865,1
8,7,20210427,2,1,0,0,0,0,0,0,954.6,1


In [75]:
all_user_date_dtf = all_user_date_dtf[:,:, dt.sort(f.user_id, f.pt_d)]

In [76]:
all_user_date_dtf

,user_id,pt_d,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,2,20210418,0,0,0,0,0,0,0,0,0,0
1,2,20210419,3,2,0,0,0,0,0,0,3455.7,1
2,2,20210420,0,0,0,0,0,0,0,0,0,0
3,2,20210421,0,0,0,0,0,0,0,0,0,0
4,2,20210422,0.666667,3,0,2,0,0,0,0,969.2,1
5,2,20210423,2.66667,6,0,1,0,0,0,0,11101,1
6,2,20210424,0,1,0,1,0,0,0,0,0,1
7,2,20210425,0,0,0,0,0,0,0,0,0,0
8,2,20210426,0,0,0,0,0,0,0,0,0,0


In [77]:
all_user_date_dtf.to_jay("./all_user_date.jay")

### 视频
>- 平均观看时长（avg watch_label）
>- 完播次数（watch_lable >= 8）
>- 被观看次数（is_watch=1）
>- 评论、收藏、分享次数
>- 退出次数（is_watch=1, watch_label=0）
>- 划过次数（is_watch=0）


In [112]:
%%time
video_all_status = df[:, {'v_avg_watch_label': mean(f.watch_label),
                           'v_sum_watch_times': sum(f.is_watch == 1),
                           'v_sum_watch_overs': sum(f.watch_label >= 8),
                           'v_sum_comment_times': sum(f.is_comment == 1),
                           'v_sum_collect_times': sum(f.is_collect == 1),
                           'v_sum_share_times': sum(f.is_share == 1),
                          'v_sum_quit_times': sum((f.watch_label == 0) & (f.is_watch == 1)),
                          'v_sum_skip_times': sum(f.is_watch == 0),
                          'v_sum_watch_days': ifelse(sum(ifelse(f.is_watch == 1, 1, 0)) > 0, 1, 0)},
                     by('video_id', 'pt_d')]

CPU times: user 5.39 s, sys: 130 ms, total: 5.51 s
Wall time: 462 ms


In [114]:
video_all_status.shape

(209448, 11)

In [110]:
# save_video_temp_data(video_all_status, 'video_all_status')

- 导出所有用户和时间的组合

In [115]:
# 保存结果
real_video_date = list(video_all_status[:, ['video_id','pt_d']].to_tuples())
all_video_date = [(v, d) for v in v_list for d in d_list] 

In [116]:
len(real_video_date)

209448

In [117]:
len(all_video_date)

330096

In [118]:
r_set = set(real_video_date)
a_set = set(all_video_date)

In [119]:
need_to_add = a_set - r_set

In [120]:
len(need_to_add)

120648

In [121]:
del r_set, a_set

In [122]:
del real_video_date, all_video_date

In [123]:
need_to_add = list(need_to_add)

In [124]:
need_to_add[0]

(27870, 20210427)

In [125]:
tmp = [(v, d) for v, d in need_to_add ]

In [126]:
video_date_dtf = dt.Frame(tmp, names=['video_id', 'pt_d'])

In [127]:
del tmp,need_to_add

In [128]:
video_date_dtf

,video_id,pt_d
,▪▪▪▪,▪▪▪▪
0,27870,20210427
1,15151,20210427
2,9565,20210418
3,9387,20210430
4,11348,20210424
5,4037,20210502
6,8088,20210502
7,24871,20210502
8,40643,20210420


In [129]:
video_all_status.names

('video_id',
 'pt_d',
 'v_avg_watch_label',
 'v_sum_watch_times',
 'v_sum_watch_overs',
 'v_sum_comment_times',
 'v_sum_collect_times',
 'v_sum_share_times',
 'v_sum_quit_times',
 'v_sum_skip_times',
 'v_sum_watch_days')

In [130]:
video_date_dtf['v_avg_watch_label'] = 0
video_date_dtf['v_sum_watch_times'] = 0
video_date_dtf['v_sum_watch_overs'] = 0
video_date_dtf['v_sum_comment_times'] = 0
video_date_dtf['v_sum_collect_times'] = 0
video_date_dtf['v_sum_share_times'] = 0
video_date_dtf['v_sum_quit_times'] = 0
video_date_dtf['v_sum_skip_times'] = 0
video_date_dtf['v_sum_watch_days'] = 0

In [131]:
video_all_status.names == video_date_dtf.names

True

In [132]:
all_video_date_dtf = dt.rbind(video_all_status, video_date_dtf, bynames = True)

In [133]:
all_video_date_dtf

,video_id,pt_d,v_avg_watch_label,v_sum_watch_times,v_sum_watch_overs,v_sum_comment_times,v_sum_collect_times,v_sum_share_times,v_sum_quit_times,v_sum_skip_times,v_sum_watch_days
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,20210419,0,4,0,0,0,0,4,0,1
1,0,20210421,0,3,0,0,0,0,3,0,1
2,0,20210422,0,4,0,0,0,0,4,0,1
3,0,20210423,2.25,4,1,0,0,0,3,0,1
4,0,20210424,3,2,0,0,0,0,0,0,1
5,0,20210425,0,3,0,0,0,0,3,0,1
6,0,20210426,4.5,2,1,0,0,0,1,0,1
7,0,20210428,0,1,0,0,0,0,1,0,1
8,0,20210429,1.25,4,0,0,0,0,3,0,1


In [134]:
all_video_date_dtf = all_video_date_dtf[:,:, dt.sort(f.video_id, f.pt_d)]

In [135]:
all_video_date_dtf

,video_id,pt_d,v_avg_watch_label,v_sum_watch_times,v_sum_watch_overs,v_sum_comment_times,v_sum_collect_times,v_sum_share_times,v_sum_quit_times,v_sum_skip_times,v_sum_watch_days
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,20210418,0,0,0,0,0,0,0,0,0
1,0,20210419,0,4,0,0,0,0,4,0,1
2,0,20210420,0,0,0,0,0,0,0,0,0
3,0,20210421,0,3,0,0,0,0,3,0,1
4,0,20210422,0,4,0,0,0,0,4,0,1
5,0,20210423,2.25,4,1,0,0,0,3,0,1
6,0,20210424,3,2,0,0,0,0,0,0,1
7,0,20210425,0,3,0,0,0,0,3,0,1
8,0,20210426,4.5,2,1,0,0,0,1,0,1


In [136]:
all_video_date_dtf.to_jay("./all_video_date.jay")

In [137]:
all_video_date_dtf

,video_id,pt_d,v_avg_watch_label,v_sum_watch_times,v_sum_watch_overs,v_sum_comment_times,v_sum_collect_times,v_sum_share_times,v_sum_quit_times,v_sum_skip_times,v_sum_watch_days
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,0,20210418,0,0,0,0,0,0,0,0,0
1,0,20210419,0,4,0,0,0,0,4,0,1
2,0,20210420,0,0,0,0,0,0,0,0,0
3,0,20210421,0,3,0,0,0,0,3,0,1
4,0,20210422,0,4,0,0,0,0,4,0,1
5,0,20210423,2.25,4,1,0,0,0,3,0,1
6,0,20210424,3,2,0,0,0,0,0,0,1
7,0,20210425,0,3,0,0,0,0,3,0,1
8,0,20210426,4.5,2,1,0,0,0,1,0,1


## 进阶统计
>根据user，video在每一天的统计信息，计算出user,video在每一天时三个周期的统计信息（1d,3d,7d）。

In [1]:
from utils import *

### 用户

In [2]:
# 加载user
# dataset/traindata/all_video_date.jay
all_user_date = dt.fread("../../dataset/traindata/all_user_date.jay").to_pandas()

In [25]:
all_user_date_groups =  all_user_date.groupby(['user_id'])

In [4]:
len(all_user_date_groups)

2690730

In [3]:
date_list = [20210419, 20210420, 20210421, 20210422, 20210423, 20210424, 20210425, 20210426, 20210427, 20210428, 20210429, 20210430, 20210501, 20210502, 20210503, 20210504]

- 1天
> 只需要将其中日期进行修改就可作为前1天的数据  
> 例如将20210418修改为20210419，则代表其为20210419前一天的数据。  
> 这里我们使用replace([原],[新])

In [4]:
all_user_date

,user_id,pt_d,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
0,2,20210418,0.000000,0,0,0,0,0,0,0,0.0,0
1,2,20210419,3.000000,2,0,0,0,0,0,0,3455.7,1
2,2,20210420,0.000000,0,0,0,0,0,0,0,0.0,0
3,2,20210421,0.000000,0,0,0,0,0,0,0,0.0,0
4,2,20210422,0.666667,3,0,2,0,0,0,0,969.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
43051675,5910798,20210429,0.000000,0,0,0,0,0,0,0,0.0,0
43051676,5910798,20210430,0.000000,0,0,0,0,0,0,0,0.0,0
43051677,5910798,20210501,0.000000,0,0,0,0,0,0,0,0.0,0
43051678,5910798,20210502,0.000000,0,0,0,0,0,0,0,0.0,0


In [5]:
all_user_date.replace([20210418, 20210419, 20210420, 20210421, 20210422, 20210423, 20210424, 20210425, 20210426, 20210427, 20210428, 20210429, 20210430, 20210501, 20210502, 20210503]
                      ,[20210419, 20210420, 20210421, 20210422, 20210423, 20210424, 20210425, 20210426, 20210427, 20210428, 20210429, 20210430, 20210501, 20210502, 20210503, 20210504], inplace=True)

In [6]:
all_user_date

,user_id,pt_d,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
0,2,20210419,0.000000,0,0,0,0,0,0,0,0.0,0
1,2,20210420,3.000000,2,0,0,0,0,0,0,3455.7,1
2,2,20210421,0.000000,0,0,0,0,0,0,0,0.0,0
3,2,20210422,0.000000,0,0,0,0,0,0,0,0.0,0
4,2,20210423,0.666667,3,0,2,0,0,0,0,969.2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
43051675,5910798,20210430,0.000000,0,0,0,0,0,0,0,0.0,0
43051676,5910798,20210501,0.000000,0,0,0,0,0,0,0,0.0,0
43051677,5910798,20210502,0.000000,0,0,0,0,0,0,0,0.0,0
43051678,5910798,20210503,0.000000,0,0,0,0,0,0,0,0.0,0


In [7]:
# 保存
all_user_date.to_csv("./user_1day.csv", index=None)

- 3天

In [ ]:
cnt = 1
all_user_3day = dt.Frame()
from time import time
start = time()

for user_id, group in all_user_date_groups:
    tmp_df = group.rolling(3, min_periods=1).agg({'u_avg_watch_label':'mean',
                                                       "u_sum_watch_times":'mean',
                                                        "u_sum_watch_overs":'mean',
                                                        "u_sum_quit_times":"mean",
                                                        "u_sum_skip_times":"mean",
                                                        "u_sum_comment_times":'mean',
                                                        "u_sum_collect_times":'mean',
                                                        "u_sum_share_times":'mean',
                                                        "u_sum_watch_time":'mean',
                                                        'u_sum_watch_days':'sum'
                                                       })
    user_id = dt.Frame(user_id = [user_id] * 16, date = date_list)
    dtf = dt.Frame(tmp_df)
    user_date_dtf = dt.cbind([user_id, dtf])
    all_user_3day = dt.rbind([all_user_3day,user_date_dtf], force=True)
#     print(all_user_3day)
    if cnt%10000 == 0:
        # 转为dataframe 追加在csv中。
        all_user_3day.to_pandas().to_csv("./tmp_user_3date.csv", index=None, mode='a')
        cost = time()-start
        print(cnt, "：追加成功，花费时间：", cost, "\n还需要 ",(2690730-cnt)/10000*cost/3600,' h')
        start = time()
        all_user_3day = dt.Frame()        
    cnt += 1
    
all_user_3day.to_pandas().to_csv("./tmp_user_3date.csv", index=None, mode='a')
print(cnt, "：追加成功。")
all_user_3day = dt.Frame()


930000 ：追加成功，花费时间： 41.95294785499573 
还需要  2.051883718797962  h
940000 ：追加成功，花费时间： 41.80826544761658 
还需要  2.033194015752938  h
950000 ：追加成功，花费时间： 41.89624762535095 
还需要  2.0258348646910322  h
960000 ：追加成功，花费时间： 41.93678379058838 
还需要  2.016145828052362  h
970000 ：追加成功，花费时间： 41.9395637512207 
还需要  2.0046295981566113  h
980000 ：追加成功，花费时间： 41.874149560928345 
还需要  1.9898712188435264  h
990000 ：追加成功，花费时间： 41.850929498672485 
还需要  1.977142536841035  h
1000000 ：追加成功，花费时间： 42.67450499534607 
还需要  2.004196273077263  h
1010000 ：追加成功，花费时间： 42.8111207485199 
还需要  1.9987204159905514  h
1020000 ：追加成功，花费时间： 42.77201271057129 
还需要  1.985013466553688  h
1030000 ：追加成功，花费时间： 42.79007530212402 
还需要  1.973965604347123  h
1040000 ：追加成功，花费时间： 42.80585861206055 
还需要  1.9628031940746309  h
1050000 ：追加成功，花费时间： 42.828020095825195 
还需要  1.9519227058839799  h
1060000 ：追加成功，花费时间： 42.85674238204956 
还需要  1.9413270973522134  h
1070000 ：追加成功，花费时间： 42.83065748214722 
还需要  1.928248097251124  h
1080000 ：追加成功，花费时间： 42.7

In [5]:
# 从 1110000 开始生成。
cnt = 1
all_user_3day = dt.Frame()
from time import time
start = time()

for user_id, group in all_user_date_groups:
    if cnt < 1110000:
        cnt += 1
        continue
    tmp_df = group.rolling(3, min_periods=1).agg({'u_avg_watch_label':'mean',
                                                       "u_sum_watch_times":'mean',
                                                        "u_sum_watch_overs":'mean',
                                                        "u_sum_quit_times":"mean",
                                                        "u_sum_skip_times":"mean",
                                                        "u_sum_comment_times":'mean',
                                                        "u_sum_collect_times":'mean',
                                                        "u_sum_share_times":'mean',
                                                        "u_sum_watch_time":'mean',
                                                        'u_sum_watch_days':'sum'
                                                       })
    user_id = dt.Frame(user_id = [user_id] * 16, date = date_list)
    dtf = dt.Frame(tmp_df)
    user_date_dtf = dt.cbind([user_id, dtf])
    all_user_3day = dt.rbind([all_user_3day,user_date_dtf], force=True)
#     print(all_user_3day)


        
    if cnt%10000 == 0:
        # 转为dataframe 追加在csv中。
        all_user_3day.to_pandas().to_csv("./tmp_user_3date_1110000.csv", index=None, mode='a')
        cost = time()-start
        print(cnt, "：追加成功，花费时间：", cost, "\n还需要 ",(2690730-cnt)/10000*cost/3600,' h')
        start = time()
        all_user_3day = dt.Frame()        
    cnt += 1
    
all_user_3day.to_pandas().to_csv("./tmp_user_3date_1110000.csv", index=None, mode='a')
print(cnt, "：追加成功。")
all_user_3day = dt.Frame()


1110000 ：追加成功，花费时间： 18.233089208602905 
还需要  0.8005997529087464  h
1120000 ：追加成功，花费时间： 41.38389849662781 
还需要  1.8056369690446723  h
1130000 ：追加成功，花费时间： 41.386152505874634 
还需要  1.7942391611248256  h
1140000 ：追加成功，花费时间： 41.65345597267151 
还需要  1.794257327236136  h
1150000 ：追加成功，花费时间： 42.06135892868042 
还需要  1.8001443761718274  h
1160000 ：追加成功，花费时间： 41.31710195541382 
还需要  1.756814652116961  h
1170000 ：追加成功，花费时间： 41.423054456710815 
还需要  1.7498133778876066  h
1180000 ：追加成功，花费时间： 41.363038063049316 
还需要  1.7357884025830692  h
1190000 ：追加成功，花费时间： 41.282800912857056 
还需要  1.7209538281653325  h
1200000 ：追加成功，花费时间： 41.37170743942261 
还需要  1.713167928643624  h
1210000 ：追加成功，花费时间： 41.45079708099365 
还需要  1.7049288544927703  h
1220000 ：追加成功，花费时间： 41.27907919883728 
还需要  1.6863994486140543  h
1230000 ：追加成功，花费时间： 41.315091133117676 
还需要  1.6763942519688608  h
1240000 ：追加成功，花费时间： 41.284419536590576 
还需要  1.6636818320643902  h
1250000 ：追加成功，花费时间： 41.49452328681946 
还需要  1.6606223481949836  h
126000

In [ ]:
%%time
new_df = all_user_date_groups.rolling(3, min_periods=1).agg({  
    'u_avg_watch_label':'mean',
                                                       "u_sum_watch_times":'mean',
                                                        "u_sum_watch_overs":'mean',
                                                        "u_sum_quit_times":"mean",
                                                        "u_sum_skip_times":"mean",
                                                        "u_sum_comment_times":'mean',
                                                        "u_sum_collect_times":'mean',
                                                        "u_sum_share_times":'mean',
                                                        "u_sum_watch_time":'mean',
                                                        'u_sum_watch_days':'sum'
                                                       })

In [ ]:
new_df.to_csv("./test.csv")

In [2]:
nf = dt.fread("./test.csv").to_pandas()

In [5]:
nf.tail()

,user_id,C0,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
12839995,1763014,12839995,4.381680e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.230039e-11,0.0
12839996,1763014,12839996,4.381680e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.230039e-11,0.0
12839997,1763014,12839997,4.381680e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.230039e-11,0.0
12839998,1763014,12839998,4.381680e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.230039e-11,0.0
12839999,1763014,12839999,4.381680e-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.230039e-11,0.0


### 视频

In [9]:
# 加载原始数据
all_video_date = dt.fread("../../dataset/traindata/all_video_date.jay").to_pandas()

In [14]:
all_video_date

,video_id,pt_d,v_avg_watch_label,v_sum_watch_times,v_sum_watch_overs,v_sum_comment_times,v_sum_collect_times,v_sum_share_times,v_sum_quit_times,v_sum_skip_times,v_sum_watch_days
0,0,20210418,0.00,0,0,0,0,0,0,0,0
1,0,20210419,0.00,4,0,0,0,0,4,0,1
2,0,20210420,0.00,0,0,0,0,0,0,0,0
3,0,20210421,0.00,3,0,0,0,0,3,0,1
4,0,20210422,0.00,4,0,0,0,0,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...
330091,50351,20210429,1.00,4,0,0,0,0,3,0,1
330092,50351,20210430,0.00,3,0,0,0,0,3,0,1
330093,50351,20210501,0.25,4,0,0,0,0,3,0,1
330094,50351,20210502,1.00,1,0,0,0,0,0,0,1


In [12]:
all_video_date_groups = all_video_date.groupby(['video_id'])

In [13]:
len(all_video_date_groups)

20631

In [15]:
date_list = [20210419, 20210420, 20210421, 20210422, 20210423, 20210424, 20210425, 20210426, 20210427, 20210428, 20210429, 20210430, 20210501, 20210502, 20210503, 20210504]

从来列表长度可以看出，只包含了被人看过的视频的，检索时如果不存在，则将其设为0。

- 1天

In [24]:
cnt = 1
all_video_1day = dt.Frame()
from time import time
start = time()

for video_id, group in all_video_date_groups:
    
    tmp_df = group.rolling(1, min_periods=1).agg({'v_avg_watch_label':'mean',
                                                   "v_sum_watch_times":'mean',
                                                    "v_sum_watch_overs":'mean',
                                                    "v_sum_comment_times":"mean",
                                                    "v_sum_collect_times":"mean",
                                                    "v_sum_share_times":'mean',
                                                    "v_sum_quit_times":'mean',
                                                    "v_sum_skip_times":'mean',
                                                    'v_sum_watch_days':'sum'
                                                   })
    
    video_id_date = dt.Frame(video_id = [video_id] * 16, date = date_list)
    dtf = dt.Frame(tmp_df)
    video_date_dtf = dt.cbind([video_id_date, dtf])
    all_video_1day = dt.rbind([all_video_1day, video_date_dtf], force=True)
#     print(all_user_3day)
    if cnt%10000 == 0:
        # 转为dataframe 追加在csv中。
        all_video_1day.to_pandas().to_csv("./tmp_video_1day.csv", index=None, mode='a')
        cost = time()-start
        print(cnt, "：追加成功，花费时间：", cost, "\n还需要 ",(20631-cnt)/10000*cost,' s')
        start = time()
        all_video_1day = dt.Frame()        
    cnt += 1
    
all_video_1day.to_pandas().to_csv("./tmp_video_1day.csv", index=None, mode='a')
print(cnt, "：追加成功。")
all_video_1day = dt.Frame()

10000 ：追加成功，花费时间： 38.11999535560608 
还需要  40.525367062544824  s
20000 ：追加成功，花费时间： 38.09775400161743 
还需要  2.40396827750206  s
20632 ：追加成功。


- 3天

In [17]:
cnt = 1
all_video_3day = dt.Frame()
from time import time
start = time()

for video_id, group in all_video_date_groups:
    
    tmp_df = group.rolling(3, min_periods=1).agg({'v_avg_watch_label':'mean',
                                                   "v_sum_watch_times":'mean',
                                                    "v_sum_watch_overs":'mean',
                                                    "v_sum_comment_times":"mean",
                                                    "v_sum_collect_times":"mean",
                                                    "v_sum_share_times":'mean',
                                                    "v_sum_quit_times":'mean',
                                                    "v_sum_skip_times":'mean',
                                                    'v_sum_watch_days':'sum'
                                                   })
    
    video_id_date = dt.Frame(video_id = [video_id] * 16, date = date_list)
    dtf = dt.Frame(tmp_df)
    video_date_dtf = dt.cbind([video_id_date, dtf])
    all_video_3day = dt.rbind([all_video_3day, video_date_dtf], force=True)
#     print(all_user_3day)
    if cnt%10000 == 0:
        # 转为dataframe 追加在csv中。
        all_video_3day.to_pandas().to_csv("./tmp_video_3days.csv", index=None, mode='a')
        cost = time()-start
        print(cnt, "：追加成功，花费时间：", cost, "\n还需要 ",(20631-cnt)/10000*cost,' s')
        start = time()
        all_video_3day = dt.Frame()        
    cnt += 1
    
all_video_3day.to_pandas().to_csv("./tmp_video_3days.csv", index=None, mode='a')
print(cnt, "：追加成功。")
all_video_3day = dt.Frame()

10000 ：追加成功，花费时间： 38.05777907371521 
还需要  2.8339619471189046  h
20000 ：追加成功，花费时间： 38.65217685699463 
还需要  2.8674868971467014  h
20632 ：追加成功。


- 7天

In [22]:
cnt = 1
all_video_7day = dt.Frame()
from time import time
start = time()

for video_id, group in all_video_date_groups:
    tmp_df = group.rolling(7, min_periods=1).agg({'v_avg_watch_label':'mean',
                                                   "v_sum_watch_times":'mean',
                                                    "v_sum_watch_overs":'mean',
                                                    "v_sum_comment_times":"mean",
                                                    "v_sum_collect_times":"mean",
                                                    "v_sum_share_times":'mean',
                                                    "v_sum_quit_times":'mean',
                                                    "v_sum_skip_times":'mean',
                                                    'v_sum_watch_days':'sum'
                                                   })
    
    video_id_date = dt.Frame(video_id = [video_id] * 16, date = date_list)
    dtf = dt.Frame(tmp_df)
    video_date_dtf = dt.cbind([video_id_date, dtf])
    all_video_7day = dt.rbind([all_video_7day, video_date_dtf], force=True)
#     print(all_user_3day)
    if cnt%10000 == 0:
        # 转为dataframe 追加在csv中。
        all_video_7day.to_pandas().to_csv("./tmp_video_7days.csv", index=None, mode='a')
        cost = time()-start
        print(cnt, "：追加成功，花费时间：", cost, "\n还需要 ",(20631-cnt)/10000*cost,' s')
        start = time()
        all_video_7day = dt.Frame()        
    cnt += 1
    
all_video_7day.to_pandas().to_csv("./tmp_video_7days.csv", index=None, mode='a',  header=None)
print(cnt, "：追加成功。")
all_video_7day = dt.Frame()

10000 ：追加成功，花费时间： 38.90198349952698 
还需要  41.35669865834713  s
20000 ：追加成功，花费时间： 38.5447793006897 
还需要  2.4321755738735202  s
20632 ：追加成功。


# test 

In [3]:
from utils import *

- user_3date_表格合并

In [4]:
user_3date_1 = dt.fread('./user_3date.csv').to_pandas()

In [5]:
user_3date_1

,user_id,date,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
0,2,20210419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,20210420,1.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1727.85,1.0
2,2,20210421,1.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,1151.8999999999999,1.0
3,2,20210422,1.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,1151.8999999999999,1.0
4,2,20210423,0.22222222222222218,1.0,0.0,0.6666666666666666,0.0,0.0,0.0,0.0,323.0666666666666,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
23200139,3184610,20210430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23200140,3184610,20210501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23200141,3184610,20210502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23200142,3184610,20210503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# 删除多的user_id header行
user_3date_1.drop(index=(user_3date_1.loc[(user_3date_1['user_id']=='user_id')].index), inplace=True)

In [8]:
user_3date_1

,user_id,date,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
0,2,20210419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,20210420,1.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1727.85,1.0
2,2,20210421,1.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,1151.8999999999999,1.0
3,2,20210422,1.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,1151.8999999999999,1.0
4,2,20210423,0.22222222222222218,1.0,0.0,0.6666666666666666,0.0,0.0,0.0,0.0,323.0666666666666,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
23200139,3184610,20210430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23200140,3184610,20210501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23200141,3184610,20210502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23200142,3184610,20210503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
user_3date_2 = dt.fread('./user_3date_1110000.csv').to_pandas()

In [10]:
# 删除多的user_id header行
user_3date_2.drop(index=(user_3date_2.loc[(user_3date_2['user_id']=='user_id')].index), inplace=True)

In [11]:
user_3date_2

,user_id,date,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
0,2437893,20210419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2437893,20210420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2437893,20210421,0.0,0.6666666666666666,0.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,1.0
3,2437893,20210422,0.0,0.6666666666666666,0.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,1.0
4,2437893,20210423,0.0,0.6666666666666666,0.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25291850,5910798,20210430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291851,5910798,20210501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291852,5910798,20210502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291853,5910798,20210503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
user_3days_all = pd.concat([user_3date_1, user_3date_2])

In [19]:
user_3days_all

,user_id,date,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
0,2,20210419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,20210420,1.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1727.85,1.0
2,2,20210421,1.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,1151.8999999999999,1.0
3,2,20210422,1.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,1151.8999999999999,1.0
4,2,20210423,0.22222222222222218,1.0,0.0,0.6666666666666666,0.0,0.0,0.0,0.0,323.0666666666666,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25291850,5910798,20210430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291851,5910798,20210501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291852,5910798,20210502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291853,5910798,20210503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# 删除重复的user_date记录
user_3days_all.drop_duplicates(subset=['user_id', 'date'],keep='first', inplace=True)

In [21]:
user_3days_all

,user_id,date,u_avg_watch_label,u_sum_watch_times,u_sum_watch_overs,u_sum_quit_times,u_sum_skip_times,u_sum_comment_times,u_sum_collect_times,u_sum_share_times,u_sum_watch_time,u_sum_watch_days
0,2,20210419,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,20210420,1.5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1727.85,1.0
2,2,20210421,1.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,1151.8999999999999,1.0
3,2,20210422,1.0,0.6666666666666666,0.0,0.0,0.0,0.0,0.0,0.0,1151.8999999999999,1.0
4,2,20210423,0.22222222222222218,1.0,0.0,0.6666666666666666,0.0,0.0,0.0,0.0,323.0666666666666,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
25291850,5910798,20210430,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291851,5910798,20210501,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291852,5910798,20210502,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25291853,5910798,20210503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
user_3days_all.to_csv("./user_3days.csv", index=None)